In [11]:
import numpy as np
from neuropy import ProcessData

basepath = "/data/Clustering/SleepDeprivation/RatN_Day1_test_neuropy"
sess = ProcessData(basepath=basepath)

## Check recording info

In [13]:
print(sess.paradigm.to_dataframe())

AttributeError: 'ProcessData' object has no attribute 'paradigm'

## Set probe configuration
Lets create probe configuration for a 128-channel probe

In [ ]:
from neuropy.core import Shank, Probe
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
for i in range(8):

    shank = Shank(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=20,
        y_shift_per_column=[0, -7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shanks.append(shank)
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)

probe1 = Probe(shanks)
sess.probegroup.add_probe(probe1)

# probe2 = Probe(shanks)
# probe2.move((probe1.x_max+400,0))
# sess.probegroup.add_probe(probe2)
sess.probegroup.save()

plot_probe(sess.probegroup)

## Create epochs for your experimental paradigm
A typical experiment involves multiple epochs such pre sleep, running on track and then another sleep epoch.

In [10]:
import pandas as pd
from neuropy.core import Epoch

epochs = pd.DataFrame(
    {
        "start": [0, 10000, 20000],
        "stop": [9000, 13600, 30000],
        "label": ["pre", "maze", "post"],
    }
)

a = Epoch(epochs=epochs)
a.filename = sess.filePrefix.with_suffix('.paradigm.npy')
a.save()



data saved


In [5]:
a['pre']

array([   0, 9000])

# Detect epochs
Here we will various types of epochs which typical for hippocampal recordings.

In [ ]:
from neuropy import analyses, plotting


## Artifacts epochs
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [ ]:
signal = sess.eegfile.get_signal([1])
artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=7)
artifact_epochs.filename = sess.filePrefix.with_suffix(".artifact.npy")
artifact_epochs.save()
plotting.plot_artifact_epochs(artifact_epochs, signal)


## Ripple epochs
To detect ripples one also needs probegroup.

In [ ]:
signal = sess.eegfile.get_signal()
ripple_epochs = analyses.detect_ripple_epochs(signal, sess.probegroup)
# ripple_epochs.filename = sess.filePrefix.with_suffix('.ripple.npy')
# ripple_epochs.save()
plotting

In [ ]:
signal = sess.eegfile.get_signal(channel_id=[1, 2, 3, 4], t_start=1, t_stop=1.2)
plotting.plot_signal_traces(signal)

# Importing spiketrains from Phy

In [ ]:
from neuropy.io import PhyIO
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

cluster_path = Path(
    "/data/Clustering/SleepDeprivation/RatN_Day1_test_neuropy/phy_clusters"
)
chan_grps = sess.recinfo.channel_groups

spk, peak_chans = [], []
for f_ind, folder in enumerate(cluster_path.iterdir()):
    phy_data = PhyIO(folder)
    spk.extend(phy_data.spiketrains)
    peak_chans.extend(chan_grps[f_ind][phy_data.peak_channels])

peak_chans = np.array(peak_chans)

neurons = Neurons(
    np.array(spk, dtype=object),
    t_stop=54000,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
)


In [ ]:
%matplotlib widget
from neuropy.plotting import plot_raster

plot_raster(neurons,color='jet',add_vert_jitter=True)